In [ ]:
# -*- coding: utf-8 -*-
"""
Created on 2023/04/11
Revised on 2023/05/30
 
@author: mjofre - Marc Jofre
e-mail: marc.jofre@upc.edu
Technical University of Catalonia - Universitat Politècnica de Catalunya (UPC)
"""
###########################################################################
# Seminar on Quantum Technologies for Cybersecurity: Networking and Systems
# Self-study
# Session 5 - Quantum Security: Networking and Systems

import os, sys, time
import numpy as np
import math
import matplotlib.pyplot as plt
from google.colab import files
import matplotlib.style
import matplotlib as mpl
#print(plt.style.available)
mpl.style.use('default')

try:  
  import qiskit
except:
  print("installing qiskit...")
  !pip install qiskit --quiet
  print("installed qiskit.")
  import qiskit


installing qiskit...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 6.5 MB/s eta 0:00:00
installed qiskit.
installing qutip...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 59.2 MB/s eta 0:00:00
installed qutip.

# Self-study (18 hours):

Read on Quantum internet (2 hours):
 - https://tu-delft.foleon.com/tu-delft/quantum-internet/the-six-stages-of-quantum-networks/
 - https://www.youtube.com/watch?v=XzPi29O6DAc
 - https://tu-delft.foleon.com/tu-delft/quantum-internet/classical-internet-vs-quantum-internet/
 - https://tu-delft.foleon.com/tu-delft/quantum-internet/the-basics-to-understand-quantum-internet/
 - https://tu-delft.foleon.com/tu-delft/quantum-internet/bringing-the-quantum-internet-to-life/

Exercise on Quantum Computing in Access Control and Network Security from [2], [11] and implement some of the python examples (6 hours).

Search and document parts of the ecosystem of quantum technologies for cybersecurity to prepare the following class session (10 hours).


In [ ]:
try:
  import sequence
  # https://github.com/sequence-toolbox/SeQUeNCe
except ImportError:
  print("installing sequence-toolbox...")
  !pip install git+https://github.com/sequence-toolbox/SeQUeNCe.git --quiet
  #!pip install sequence_toolbox --quiet
  print("installed sequence-toolbox.")
  import sequence

In [ ]:
# https://github.com/sequence-toolbox/SeQUeNCe/blob/master/example/three_node_eg_ep_es.ipynb

from ipywidgets import interact
from matplotlib import pyplot as plt
import time

from sequence.kernel.timeline import Timeline
from sequence.topology.node import QuantumRouter, BSMNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel

def test(sim_time, cc_delay, qc_atten, qc_dist):
  """
  sim_time: duration of simulation time (ms)
  cc_delay: delay on classical channels (ms)
  qc_atten: attenuation on quantum channels (dB/m)
  qc_dist: distance of quantum channels (km)
  """
  
  PS_PER_MS = 1e9
  M_PER_KM = 1e3
  
  # convert units for cc delay (to ps) and qc distance (to m)
  cc_delay *= PS_PER_MS
  qc_dist *= M_PER_KM
  
  raw_fidelity = 0.85
  
  # construct the simulation timeline; the constructor argument is the simulation time (in ps)
  tl = Timeline(sim_time * PS_PER_MS)
  
  ## create our quantum network and update parameters as needed
  
  # first, construct the quantum routers
  # (with arguments for the node name, timeline, and number of quantum memories)
  r1 = QuantumRouter("r1", tl, 50)
  r2 = QuantumRouter("r2", tl, 100)
  r3 = QuantumRouter("r3", tl, 50)
  # next, construct the BSM nodes
  # (with arguments for the node name, timeline, and the names of connected routers)
  m1 = BSMNode("m1", tl, ["r1", "r2"])
  m2 = BSMNode("m2", tl, ["r2", "r3"])
  
  r1.add_bsm_node(m1.name, r2.name)
  r2.add_bsm_node(m1.name, r1.name)
  r2.add_bsm_node(m2.name, r3.name)
  r3.add_bsm_node(m2.name, r2.name)
  
  # set seeds for random generators
  nodes = [r1, r2, r3, m1, m2]
  for i, node in enumerate(nodes):
    node.set_seed(i)
  
  for node in [r1, r2, r3]:
    memory_array = node.get_components_by_type("MemoryArray")[0]
    # we update the coherence time (measured in seconds) here
    memory_array.update_memory_params("coherence_time", 10)
    # and similarly update the fidelity of entanglement for the memories
    memory_array.update_memory_params("raw_fidelity", raw_fidelity)
  
  # create all-to-all classical connections
  for node1 in nodes:
    for node2 in nodes:
      if node1 == node2:
        continue
      # construct a classical communication channel
      # (with arguments for the channel name, timeline, length (in m), and delay (in ps))
      cc = ClassicalChannel("cc_%s_%s"%(node1.name, node2.name), tl, 1e3, delay=cc_delay)
      cc.set_ends(node1, node2.name)
          
  # create quantum channels linking r1 and r2 to m1
  # (with arguments for the channel name, timeline, attenuation (in dB/km), and distance (in m))
  qc0 = QuantumChannel("qc_r1_m1", tl, qc_atten, qc_dist)
  qc1 = QuantumChannel("qc_r2_m1", tl, qc_atten, qc_dist)
  qc0.set_ends(r1, m1.name)
  qc1.set_ends(r2, m1.name)
  # create quantum channels linking r2 and r3 to m2
  qc2 = QuantumChannel("qc_r2_m2", tl, qc_atten, qc_dist)
  qc3 = QuantumChannel("qc_r3_m2", tl, qc_atten, qc_dist)
  qc2.set_ends(r2, m2.name)
  qc3.set_ends(r3, m2.name)

  # create routing table manually
  # note that the routing table is based on quantum links, not classical
  # the arguments are the names of the destination node and the next node in the best path towards the destination
  r1.network_manager.protocol_stack[0].add_forwarding_rule("r2", "r2")
  r1.network_manager.protocol_stack[0].add_forwarding_rule("r3", "r2")
  r2.network_manager.protocol_stack[0].add_forwarding_rule("r1", "r1")
  r2.network_manager.protocol_stack[0].add_forwarding_rule("r3", "r3")
  r3.network_manager.protocol_stack[0].add_forwarding_rule("r1", "r2")
  r3.network_manager.protocol_stack[0].add_forwarding_rule("r2", "r2")
  
  ## run our simulation
  
  tl.init()
  # we use the network manager of an end router to make our entanglement request
  # here, the arguments are:
  # (1) the destination node name,
  # (2) the start time (in ps) of entanglement,
  # (3) the end time (in ps) of entanglement,
  # (4) the number of memories to entangle, and
  # (5) the desired fidelity of entanglement.
  r1.network_manager.request("r3", 1e12, 1e14, 50, 0.9)

  tick = time.time()
  tl.run()
  print("execution time %.2f sec" % (time.time() - tick))
  
  ## display metrics for entangled memories
  
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
  fig.set_size_inches(12, 5)

  # entangled memories on r1
  # here, we plot the number of entangled memories versus time for r1
  data = []
  for info in r1.resource_manager.memory_manager:
    if info.entangle_time > 0:
      data.append(info.entangle_time / 1e12)
  data.sort()
  ax1.plot(data, range(1, len(data) + 1), marker="o")
  ax1.set_title("r1")
  ax1.set_ylabel("Number of Entangled Memories")
  
  # entangled memories on r2
  data = []
  for info in r2.resource_manager.memory_manager:
    if info.entangle_time > 0:
      data.append(info.entangle_time / 1e12)
  data.sort()
  ax2.plot(data, range(1, len(data) + 1), marker="o")
  ax2.set_title("r2")
  ax2.set_xlabel("Simulation Time (s)")
  
  # entangled memories on r3
  data = []
  for info in r3.resource_manager.memory_manager:
    if info.entangle_time > 0:
      data.append(info.entangle_time / 1e12)
  data.sort()
  ax3.plot(data, range(1, len(data) + 1), marker="o")
  ax3.set_title("r3")
  
  fig.tight_layout()

  ## display metrics for memory fidelities
  
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
  fig.set_size_inches(12, 5)
  
  # display collected metric for memory fidelities on r1
  # in this case, a bar chart of memory fidelity at each index
  data = []
  for info in r1.resource_manager.memory_manager:
    data.append(info.fidelity)
  ax1.bar(range(len(data)), data)
  ax1.plot([0, len(data)], [raw_fidelity, raw_fidelity], "k--")
  ax1.plot([0, len(data)], [0.9, 0.9], "k--")
  ax1.set_ylim(0.7,1)
  ax1.set_title("r1")
  ax1.set_ylabel("Fidelity")

  # display collected metric for memory fidelities on r2
  data = []
  for info in r2.resource_manager.memory_manager:
    data.append(info.fidelity)
  ax2.bar(range(len(data)), data)
  ax2.plot([0, len(data)], [raw_fidelity, raw_fidelity], "k--")
  ax2.plot([0, len(data)], [0.9, 0.9], "k--")
  ax2.set_ylim(0.7,1)
  ax2.set_title("r2")
  ax2.set_xlabel("Memory Number")

  # display collected metric for memory fidelities on r3
  data = []
  for info in r3.resource_manager.memory_manager:
    data.append(info.fidelity)
  ax3.bar(range(len(data)), data)
  ax3.plot([0, len(data)], [raw_fidelity, raw_fidelity], "k--")
  ax3.plot([0, len(data)], [0.9, 0.9], "k--")
  ax3.set_ylim(0.7,1)
  ax3.set_title("r3")
  
  fig.tight_layout()

interactive_plot = interact(test, sim_time=(2000, 4000, 500), cc_delay=(0.1, 1, 0.1), qc_atten=[1e-5, 2e-5, 3e-5], qc_dist=(1, 10, 1))
interactive_plot

In [ ]:
# https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/awards/teach_me_qiskit_2018/chsh_game/CHSH%20game-tutorial.ipynb
# useful packages 
import random as rand

# importing Qiskit
from qiskit import BasicAer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

def cplayer_output(strategy, inp):
  if(strategy == 1):
      return inp    
  elif(strategy == 2):
      return abs(inp-1)    
  elif(strategy == 3):
      return 1        
  elif(strategy == 4):
      return 0            
  else:
      print("INVALID choice")
      return 100

# Pick Alice's classical strategy
A_st = int(input('select the classical strategy for Alice, input 1,2,3 or 4 to pick one of the strategies listed above '))

# Pick Bob's classical strategy
B_st = int(input('select the classical strategy for Bob, input 1,2,3 or 4 to pick one of the strategies listed above '))

# fixes the numbers of games to be played
N=100

# initializes counters used to keep track of the numbers of games won and played by Alice an Bob
cont_win = 0 # counts games won
cont_tot = 0 # counts games played

# play the game N times
for i in range(N):    
  # generates two random input from the refree, x and y, to be given to Alice and Bob
  random_num1 = rand.random() # first random number
  random_num2 = rand.random() # second random number

  if(random_num1 >= 1/2): # converts the first random number to 0 or 1
      x = 0
  else: x = 1

  if(random_num2 >= 1/2): # converts the second random number to 0 or 1
      y = 0
  else: y = 1    
  
  # generates Alice's and Bob's output
  a = cplayer_output(A_st, x) # Alice's output    
  b = cplayer_output(B_st, y) # Bob's output
  # check if the condition for winning the game is met
  if(x*y == a^b):
      cont_win += 1 # increase thes won games' counter if the condition to win the game is met    
  cont_tot += 1 # increases the played games' counter

Prob_win = cont_win/cont_tot # winning probability

print('Alice and Bob won the game with probability: ', Prob_win*100, '%')

def qAlice_output(strategy, inp):
  if(strategy == 1):
      return 0    
  elif(strategy == 2):
      return rand.uniform(0,2*np.pi)    
  elif(strategy == 3):
    if(inp == 0):
        return 0
    elif(inp == 1):
        return np.pi/2            
  else:
      print("INVALID choice")
      return 100

def qBob_output(strategy, inp):
  if(strategy == 1):
      return 0    
  elif(strategy == 2):
      return rand.uniform(0,2*np.pi)    
  elif(strategy == 3):
    if(inp == 0):
        return np.pi/4
    elif(inp == 1):
        return -np.pi/4            
  else:
      print("INVALID choice")
      return 100

# Alice's strategy
qA_st = int(input('select the quantum strategy for Alice, input 1,2 or 3 to pick one of the strategies listed above: '))

# Bob's strategy
qB_st = int(input('select the quantum strategy for Bob, input 1,2 or 3 to pick one of the strategies listed above: '))

# set parameters of the quantum run of the game 
shots = 1 # set how many times the circuit is run, accumulating statistics about the measurement outcomes 
backend = BasicAer.get_backend('qasm_simulator') # set the machine where the quantum circuit is to be run   

#fixes the numbers of games to be played
N=100

# initializes counters used to keep track of the numbers of games won and played by Alice an Bob
cont_win = 0 # counts games won
cont_tot = 0 # counts games played

#play N games
for i in range(N):
  # creates registers for qubits and bits
  # creates a quantum register, it specifies the qubits which are going to be used for the program
  q = QuantumRegister(2, name='q') 
  # creates a classical register, the results of the measurement of the qubits are stored here
  c = ClassicalRegister(2, name='c') 

  # creates quantum circuit, to write a quantum algorithm we will add gates to the circuit
  game = QuantumCircuit(q, c, name='game')
  
  # These gates prepare the entangled Bell pair to be shared by Alice and Bob as part of their quantum strategy
  # Alice will have qubit 0 and Bob will have qubit 1
  game.h(q[0]) # Hadamard gate on qubit 0
  game.cx(q[0],q[1]) # CNOT gate on qubit 1 controlled by qubit 0

  # generates two random input from the refree, x and y, to be given to Alice and Bob
  random_num1 = rand.random() # first random number
  random_num2 = rand.random() # second random number

  if(random_num1 >= 1/2): # converts the first random number to 0 or 1
    x = 0
  else: x = 1

  if(random_num2 >= 1/2): # converts the second random number to 0 or 1
    y = 0
  else: y = 1

  # The main part of Alice and Bob quantum strategy is to fix different rotation angles for their qubit according to the input x,y
  theta = qAlice_output(qA_st, x) # fixes Alice's rotation for her qubit
  phi = qBob_output(qB_st, y) # fixes Bob's rotation for his qubit
  
  # The following gates rotate Alice's qubit and Bob's qubit
  game.ry(theta,q[0]) #rotates Alice's qubit of an angle theta
  game.ry(phi,q[1]) ##rotates Bob's qubit of an angle phi

  # These gates are used to measure  the value of the qubits
  game.measure(q[0], c[0]) # measure Alice's qubit and stores the result in a classical bit
  game.measure(q[1], c[1]) # measure Bob's qubit and stores the result in a classical bit

  # executes circuit and store the output of the measurements
  result = execute(game, backend=backend, shots=shots).result()

  data = result.get_counts('game') # extract the outcomes and their statistics from the result of the execution
  
  # reads the result of the measurements of the quantum system
  for outcomes in data.keys():
    out = outcomes

  # converts the result of the measurements contained in the classical register as string '00', '01', '10', '11',
  # which are the answers of Alice(a) and Bob (b), from a 'string' type  to 'integer' type 
  if(out == '00'):
    a = 0
    b = 0
  if(out == '01'):
    a = 1
    b = 0    
  if(out == '10'):
    a = 0
    b = 1
  if(out == '11'):
    a = 1
    b = 1

  # check if the condition for winning the game is met
  if(x*y == a^b):
    cont_win += 1 # increase thes won games' counter if the condition to win the game is met
  
  cont_tot += 1 # increases the played games' counter

qProb_win = cont_win/cont_tot

print('Alice and Bob won the game with probability: ', qProb_win*100, '%')

if Prob_win > qProb_win :
  print("The classical strategy gave Alice and Bob higher chances of winning")
else:
  print("The quantum strategy gave Alice and Bob higher chances of winning")